In [1]:
#Hiding the codes from the cells, so that only the outputs are visible
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

In [1]:
#Importing required libraries
from ipywidgets import widgets
import io
import pandas as pd

#Creating the upload widgets so that required files can be uploaded. This section can be modified in accordance with the data provided
uploader = widgets.FileUpload(description = 'astrosat')
display(uploader)
uploaderOne = widgets.FileUpload(description = 'hmxbcat')
display(uploaderOne)
uploaderTwo = widgets.FileUpload(description = 'lmxbcat')
display(uploaderTwo)

FileUpload(value={}, description='astrosat')

FileUpload(value={}, description='hmxbcat')

FileUpload(value={}, description='lmxbcat')

In [2]:
import pandas as pd
from astropy.table import Table #For passing the catalogs to the ipyaladin widget

#Loading the uploaded files into a dataframe
df = pd.read_csv(io.BytesIO(uploader.value[next(iter(uploader.value))]['content']))
dfOne = pd.read_csv(io.BytesIO(uploaderOne.value[next(iter(uploaderOne.value))]['content']))
dfTwo = pd.read_csv(io.BytesIO(uploaderTwo.value[next(iter(uploaderTwo.value))]['content']))


# Converting the catalogs in catalog A to astropy tables. These would be visualised later
tableOne = Table.from_pandas(dfOne)
tableTwo = Table.from_pandas(dfTwo)

#Converting the coordinates in dfOne(Catalog A, hmxb) to degrees
Ra= []
Dec = []
for i in range(len(dfOne)):
    a = list(dfOne['RAJ2000'][i].split())
    Ra.append(float(a[0])*15+float(a[1])*(15/60)+float(a[2])*(15/3600))
    b = list(dfOne['DEJ2000'][i].split())
    Dec.append(float(b[0])+float(b[1])*(1/60)+float(b[2])*(1/3600)) 
dfOne['Ra'] = Ra
dfOne['Dec'] = Dec

#Converting the coordinates in dfTwo(Catalog B, lmxb) to degrees
Ra= []
Dec = []
for i in range(len(dfTwo)):
    a = list(dfTwo['RAJ2000'][i].split())
    Ra.append(float(a[0])*15+float(a[1])*(15/60)+float(a[2])*(15/3600))
    b = list(dfTwo['DEJ2000'][i].split())
    Dec.append(float(b[0])+float(b[1])*(1/60)+float(b[2])*(1/3600)) 
dfTwo['Ra'] = Ra
dfTwo['Dec'] = Dec

#Finding which elements in the catalogs A(hmxb and lmxb) and catalog B (astrosat) are common
listOne = []
listTwo = []
epsilon = 0.1 #Allowed tolerance for variation in RA, Dec values in J2000 (checked and verified with other values, and comparision)
for i in range(len(df)): #Finding the correlations between hmxb and astrosat files
    for j in range(len(dfOne)):
        if (abs(dfOne['Ra'][j]-df['CR1'][i])<epsilon) and (abs(dfOne['Dec'][j]-df['CR2'][i])<epsilon):
            listOne.append([j, i])
for i in range(len(df)): #Finding the correlations between lmxb and astrosat files
    for j in range(len(dfTwo)):
        if (abs(dfTwo['Ra'][j]-df['CR1'][i])<epsilon) and (abs(dfTwo['Dec'][j]-df['CR2'][i])<epsilon):
            listTwo.append([j,i])
            
#Loading and displaying catalog A with ipyaladin widget            
import ipyaladin as ipyal
aladin = ipyal.Aladin(target='messier 51', fov=1)
display(aladin)
aladin.add_table(tableOne)
aladin.add_table(tableTwo)

Aladin(fov=1.0, options=['allow_full_zoomout', 'coo_frame', 'fov', 'full_screen', 'log', 'overlay_survey', 'ov…

In [5]:
#Widgets for searching for objects by name, and downloading a pdf
from ipywidgets import widgets
import fpdf

#Search the object
text = widgets.Text(placeholder = 'Enter the name of the object')
display(text)
def search(placeholder):
    if (dfOne['Name']== text.value).any(): #Print the pertinent row in the dataframe if object found
        print('From the high mass x ray binary catalogue\n\n')
        content = dfOne[dfOne['Name']==text.value]
        for i in content.iterrows():
            ind = i[0] #Locating the rows in the hmxb catalog
            print(i)
        for i in range(len(listOne)): #Check for correlations in hmxb and astrosat and print if object found in astrosat
            if listOne[i][0]==ind:
                print('\nFrom the Astrosat catalogue\n')
                contentA = df.iloc[listOne[i][1], :]
                print(contentA)
                
            
            
            
    elif (dfTwo['Name']==text.value).any(): #Print the pertinent row in the dataframe if object found
        print('From the low mass x ray binary catalogue\n\n')
        content = dfTwo[dfTwo['Name']==text.value]
        for i in content.iterrows():
            ind = i[0] #Locating the rows in the lmxb catalog
            print(i)
        for i in range(len(listTwo)): #Check for correlations in lmxb and astrosat and print if object found in astrosat
            if listTwo[i][0]==ind:
                print('\nFrom the Astrosat catalogue\n')
                contentB = df.iloc[listTwo[i][1], :]
                print(contentB)
    else:
        print('Object not found in the database uploaded') #In case the user enters an incorrect name
        
        
        
#printing the displayed information
def download(placeholder):
    
    pdf = fpdf.FPDF(format='letter')
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    
    
    if (dfOne['Name']== text.value).any():
        pdf.write(5, 'From the high mass x ray binary catalogue\n\n')
        content = dfOne[dfOne['Name']==text.value]
        for i in content.iterrows():
            ind = i[0] #Locating the rows in the hmxb catalog
            pdf.write(5, str(i))
            pdf.ln()
        for i in range(len(listOne)):
            if listOne[i][0]==ind:
                pdf.write(5, '\nFrom the Astrosat catalogue\n')
                contentA = df.iloc[listOne[i][1], :]
                pdf.write(5, str(contentA))
                
            
            
            
    elif (dfTwo['Name']==text.value).any():
        pdf.write(5, 'From the low mass x ray binary catalogue\n\n')
        content = dfTwo[dfTwo['Name']==text.value]
        for i in content.iterrows():
            ind = i[0] #Locating the rows in the lmxb catalog
            pdf.write(5, str(i))
            pdf.ln()
        for i in range(len(listTwo)):
            if listTwo[i][0]==ind:
                pdf.write(5, '\nFrom the Astrosat catalogue\n')
                contentB = df.iloc[listTwo[i][1], :]
                pdf.write(5, str(contentB))
    else:
        print('Object not found in the database uploaded')

    pdf.output(text.value+'.pdf')
        
#Buttons and search bar widgets        
btn = widgets.Button(description = 'search')
display(btn)
btn.on_click(search)
btnOne = widgets.Button(description = 'Download (PDF)')
display(btnOne)
btnOne.on_click(download)



Text(value='', placeholder='Enter the name of the object')

Button(description='search', style=ButtonStyle())

Button(description='Download (PDF)', style=ButtonStyle())

From the high mass x ray binary catalogue


(57, Name                                                J0635+0533
Type                                                         P
RAJ2000                                           06 35 17.400
u_RAs                                                      NaN
DEJ2000                                           +05 33 20.90
u_DEs                                                      NaN
GLON                                                     206.1
GLAT                                                        -1
Pos                                                          x
e_Pos                                                       60
Opt                                                        NaN
r_Opt                                                      352
Vmag                                                     12.83
Vmagl                                                      NaN
u_Vmag                                                     NaN
B-V   